# Imports

In [14]:
import pandas as pd
import numpy as np
from scipy.stats import laplace
import random
import matplotlib.pyplot as plt
from scipy.integrate import quad

df_adult = pd.read_csv("adult.csv")

# Ploting Laplace Distribution

In [15]:
def plota(scale, a, b):
    dist = laplace(0, scale)
    x = np.linspace(-100, 100,1000)
    plt.plot(x, dist.pdf(x), c='black')
    plt.show()

In [16]:
def consulta1(df,eplison):
    if(eplison == 0): return np.mean(df['age'].values)
    sensibilidade = np.mean(df['age'].values)-(np.sum(df['age'].values)-np.max(df['age'].values))/(df.shape[0]-1)
    x = laplace.rvs(loc = 0, scale = sensibilidade/eplison)
    return [np.mean(df['age'].values) + x,quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0],sensibilidade]

In [17]:
def consulta2(df,eplison):
    if(eplison == 0): return np.sum(df['capitalgain'].values)
    sensibilidade = np.max(df['capitalgain'].values)
    x = laplace.rvs(loc = 0, scale = sensibilidade/eplison)
    return [np.sum(df['capitalgain'].values) + x,quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0],sensibilidade]

In [18]:
def consulta3(df,eplison):
    if(eplison == 0): return np.mean(df['hoursperweek'].values)
    sensibilidade = np.mean(df['hoursperweek'].values)-(np.sum(df['hoursperweek'].values)-np.max(df['hoursperweek'].values))/(df.shape[0]-1)
    x = laplace.rvs(loc = 0, scale = sensibilidade/eplison)
    return [np.mean(df['hoursperweek'].values) + x,quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0],sensibilidade]

In [19]:
def consulta4(df,eplison):
    df_copy = df.copy()
    df_copy = df_copy.replace('>50K', 1)
    df_copy = df_copy.replace('<=50K', 0)
    df_copy['class'] = df_copy['class'].astype(int)
    if(eplison == 0): return np.sum(df_copy['class'].values)
    sensibilidade = np.max(df_copy['class'].values) 
    x = laplace.rvs(loc = 0, scale = sensibilidade/eplison)
    return [np.sum(df_copy['class'].values) + x,quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0],sensibilidade]

In [20]:
def consulta1_teste(df, res,eplison):
    sensibilidade = np.mean(df['age'].values)-(np.sum(df['age'].values)-np.max(df['age'].values))/(df.shape[0]-1)
    x = res-np.mean(df_adult['age'].values)
    return quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]
    

In [21]:
def consulta2_teste(df, res,eplison):
    sensibilidade = np.max(df['capitalgain'].values)
    x = res-np.sum(df_adult['capitalgain'].values)
    return quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]
    

In [22]:
def consulta3_teste(df, res,eplison):
    sensibilidade = np.mean(df['hoursperweek'].values)-(np.sum(df['hoursperweek'].values)-np.max(df['hoursperweek'].values))/(df.shape[0]-1)
    x = res-np.mean(df_adult['hoursperweek'].values)
    return quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]

In [23]:
def consulta4_teste(df, res,eplison):
    df_copy = df_adult.copy()
    df_copy = df_copy.replace('>50K', 1)
    df_copy = df_copy.replace('<=50K', 0)
    df_copy['class'] = df_copy['class'].astype(int)
    df_copy2 = df.copy()
    df_copy2 = df_copy2.replace('>50K', 1)
    df_copy2 = df_copy2.replace('<=50K', 0)
    df_copy2['class'] = df_copy2['class'].astype(int)
    sensibilidade = np.max(df_copy2['class'].values) 
    x = res-np.sum(df_copy['class'].values)
    return quad(laplace.pdf,x-0.0000000001, x+0.0000000001, args = (0,sensibilidade/eplison))[0]

In [29]:
def testa_diffpriv(eplison):
    r = random.randrange(0,df_adult.shape[0]-1)
    df_vizinho = df_adult.copy()
    df_vizinho = df_vizinho.drop(df_vizinho.index[r])
    c1 = consulta1(df_adult,eplison)
    c2 = consulta2(df_adult,eplison)
    c3 = consulta3(df_adult,eplison)
    c4 = consulta4(df_adult,eplison)
    if(
        np.log(c1[1]/consulta1_teste(df_vizinho, c1[0], eplison)) <= eplison and
        np.log(c2[1]/consulta2_teste(df_vizinho, c2[0], eplison)) <= eplison and
        np.log(c3[1]/consulta3_teste(df_vizinho, c3[0], eplison)) <= eplison and
        np.log(c4[1]/consulta4_teste(df_vizinho, c4[0], eplison)) <= eplison
    ):
        return True
    
    print(r)
    print(c1[0])
    print(c1[1])
    print(consulta1_teste(df_vizinho,c1[0],eplison))
    return False
        

In [ ]:
#k=1
#while(k>0):
#    if(not testa_diffpriv(0.1)):break
#    print("ok")

In [26]:
#laplace.pdf(-0.01,0,0.0045)

# Parameters

In [13]:
eplisons = [0.025,0.25,2.5]

# Consulta 1 

In [14]:
c1_results = []
c1_sens = []

for eplison in eplisons:
    c1 = consulta1(df_adult,eplison)
    c1_results.append(c1[0])
    c1_sens.append(c1[2])

# Consulta 2

In [15]:
c2_results = []
c2_sens = []

for eplison in eplisons:
    c2 = consulta2(df_adult,eplison)
    c2_results.append(c2[0])
    c2_sens.append(c2[2])

# Consulta 3

In [16]:
c3_results = []
c3_sens = []

for eplison in eplisons:
    c3 = consulta3(df_adult,eplison)
    c3_results.append(c3[0])
    c3_sens.append(c3[2])

# Consulta 4

In [17]:
c4_results = []
c4_sens = []

for eplison in eplisons:
    c4 = consulta4(df_adult,eplison)
    c4_results.append(c4[0])
    c4_sens.append(c4[2])

In [18]:
data = []
budgets = [0.1,1,10]
i = 0
for budget in budgets:
    data.append([budget,c1_results[i],c2_results[i],c3_results[i],c4_results[i],c1_sens[i],c2_sens[i],c3_sens[i],c4_sens[i]])
    i += 1

In [19]:
results = pd.DataFrame(data, columns = [ 'budget', 'result q1', 'result q2', 'result q3', 'result q4',
                                        'sens q1', 'sens q2', 'sens q3', 'sens q4']) 

In [20]:
results

,budget,result q1,result q2,result q3,result q4,sens q1,sens q2,sens q3,sens q4
0,0.1,1.771940,9762.793918,1.950605,11709.251837,0.000046,4,0.000042,1
1,1.0,1.771022,9793.222573,1.950959,11684.782955,0.000046,4,0.000042,1
2,10.0,1.771059,9784.778068,1.950695,11687.814657,0.000046,4,0.000042,1


In [21]:
results.to_csv('result.csv',index=False,sep=',')